#  MLlib 파이프라인 & 분류 베이스라인
- Estimator vs Transformer, Pipeline, Fit/Transform
- StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler
- Logistic Regression 베이스라인 구축
- 평가 지표: AUC, PR, Confusion Matrix
- 산출물: 공통 파이프라인 템플릿 코드, baseline 결과표

## 1. 데이터 로드 및 확인

In [ ]:
# CSV 파일을 DataFrame으로 읽기
# 상위 3개 행 출력 (데이터 확인용)

In [ ]:
# 데이터 스키마 확인

## 2. MLlib 파이프라인 구조

### Transformer vs Estimator
- **Transformer**: transform() 메서드로 DataFrame을 변환 (예: VectorAssembler)
- **Estimator**: fit() 메서드로 학습 후 Transformer 생성 (예: StringIndexer, StandardScaler)

### 단계별 구성
1. **StringIndexer**: 범주형 컬럼(Gender) → 숫자 인덱스 (fit으로 vocabulary 결정)
2. **OneHotEncoder**: 인덱스 → 희소 벡터 (원-핫 인코딩)
3. **VectorAssembler**: 여러 컬럼 → 단일 feature 벡터 (Transformer)
4. **StandardScaler**: 평균 0, 분산 1로 정규화 (fit으로 mean/std 결정)

In [ ]:
# 1. StringIndexer: 문자열을 숫자로 변환 (Gender → Gender_idx)
# 예: Male → 0, Female → 1
# 2. OneHotEncoder: 범주형 숫자를 원-핫 벡터로 변환 (Gender_idx → Gender_ohe)
# 예: 0 → [1, 0], 1 → [0, 1]
# 3. VectorAssembler: 여러 피처를 하나의 벡터로 결합 (features 컬럼 생성)
# 예: [19, 19000, [1,0]] → [19.0, 19000.0, 1.0, 0.0]
# 4. StandardScaler: 피처를 표준화 (평균 0, 분산 1로 스케일링)

### Pipeline으로 순서 정의
- 모든 단계를 Pipeline에 넣으면 fit() 한 번에 전 단계 학습
- transform()으로 새 데이터에도 동일한 변환 적용

In [ ]:
# Pipeline 생성: 전처리 단계들을 하나로 묶음
# 순서: StringIndexer → OneHotEncoder → VectorAssembler → StandardScaler
# Pipeline 학습: 각 단계를 순차적으로 학습
# 내부 동작:
# 1. indexer.fit(df) → Gender 고유값 파악 (Male, Female)
# 2. encoder는 학습 불필요 (Transformer)
# 3. assembler는 학습 불필요 (Transformer)
# 4. scaler.fit() → features의 평균, 표준편차 계산
# 학습된 Pipeline으로 데이터 변환
# 내부 동작:
# 1. indexer_model.transform(df) → Gender를 숫자로 변환
# 2. encoder.transform() → 숫자를 원-핫 벡터로 변환
# 3. assembler.transform() → 모든 피처를 하나의 벡터로 결합
# 4. scaler_model.transform() → 학습된 평균/표준편차로 표준화

In [ ]:
# 변환 결과 확인 (원본 데이터와 최종 변환된 피처 비교)

## 3. 분류 베이스라인 구축 (Logistic Regression)

### Train/Test 분리

In [ ]:
# 파이프라인 적용 후 ML 모델 학습용 데이터 준비
# 파이프라인으로 전처리 수행 (학습 + 변환)
# ML 모델에 필요한 컬럼만 선택 및 이름 변경
# - scaled_features: 표준화된 피처 벡터
# - Purchased: 타겟 변수 (예측할 값, 0 또는 1)
# - withColumnRenamed: 컬럼명을 "scaled_features" → "features"로 변경
#   (PySpark ML 모델은 기본적으로 "features" 컬럼명을 기대함)

In [ ]:
# Train/Test split (80:20, seed 고정)

### Logistic Regression 모델 학습

In [ ]:
# Test 데이터에 예측
# 예측 결과 확인

### 평가 지표 계산

- **AUC (ROC)**: ROC 곡선 아래 면적 (0~1, 높을수록 좋음)
- **AUC (PR)**: Precision-Recall 곡선 아래 면적
- **Accuracy**: 정확도 (맞춘 비율)

In [ ]:
# 이진 분류 평가 지표 설정 및 계산
# 1. AUC-ROC 평가기 생성 (ROC 곡선 아래 면적)
# 2. AUC-PR 평가기 생성 (Precision-Recall 곡선 아래 면적)
# 3. 정확도 평가기 생성
# 평가 지표 계산

### Confusion Matrix

In [ ]:
# 실제 vs 예측 분포

## 정리

**학습한 내용:**
1. MLlib 파이프라인 구조 (Transformer/Estimator)
2. 전처리 단계 (StringIndexer, OneHotEncoder, VectorAssembler, StandardScaler)
3. Logistic Regression 베이스라인 모델 구축
4. 평가 지표 (AUC, PR, Accuracy, Confusion Matrix)

**다음 단계:**
- 050_CrossValidator_Tuning.py에서 하이퍼파라미터 튜닝으로 성능 개선